# DAR

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(repr)
    library(patchwork)
    library(ggplot2)
    library(Signac)
    library(tidyverse)
    library(GenomicRanges)
    library(edgeR)
    library(SingleCellExperiment)
    library(Matrix)
    library(scran)
    library(scater)
    library(ggrepel)
    library(fs)
})
options(future.globals.maxSize = Inf)
options(Seurat.object.assay.version = "v5")
options(ggrepel.max.overlaps = Inf)

In [2]:
data_dir <- "/tscc/projects/ps-epigen/users/biy022/biccn/data/SNAREdata/cell_peak_matrix/"
root_dir <- "/tscc/projects/ps-epigen/users/biy022/biccn/data/SNAREdata/downsample_analysis/"
setwd(root_dir)
output_dir <- "region_dars"

In [3]:
meta_file <- "../cell_gene_matrix/20230313_RNA_metadata.xls"
meta_df <- read.table(meta_file, sep = "\t", header = T, row.names = 1)

In [9]:
for (file in dir_ls(data_dir, glob = "*rds")) {
    subclass <- path_ext_remove(path_file(file))
    if (subclass == "SST_CHODL") {
        next
    }
    if (subclass == "Micro_PVM") {
        subclass <- "Micro"
    }
    print(subclass)
    
    subclass_result_dir <- sprintf("%s/%s/", output_dir, subclass)
    if (!dir_exists(subclass_result_dir)) {
        dir_create(subclass_result_dir)
    }
    
    subclass_matrix_file <- file
    subclass_matrix <- readRDS(subclass_matrix_file)
    subclass_down_barcodes <- read.table(sprintf("sample_barcodes/%s.tsv", subclass), sep = "\t", header = T)$barcode
    subclass_matrix <- subclass_matrix[, subclass_down_barcodes]
    subclass_meta <- meta_df[colnames(subclass_matrix), ]
    
    subclass_meta <- subclass_meta[match(colnames(subclass_matrix), rownames(subclass_meta)), ]
    subclass_meta$region_donor <- paste(subclass_meta$Region, subclass_meta$PatientID, sep = "_")
    transposed_matrix <- t(subclass_matrix)
    summed_counts <- rowsum(transposed_matrix, group = subclass_meta$region_donor)
    subclass_matrix_aggr <- t(summed_counts)
    
    group_info <- do.call(rbind, strsplit(colnames(subclass_matrix_aggr), "_"))
    group_info <- data.frame(group_info)
    colnames(group_info) <- c("region", "patient")
    rownames(group_info) <- colnames(subclass_matrix_aggr)
    
    group <- factor(group_info$region)
    y <- DGEList(counts = subclass_matrix_aggr, group = group, remove.zeros = TRUE)
    keep <- filterByExpr(y, min.count = 5, min.total.count = 15)
    y <- y[keep, , keep.lib.sizes=FALSE]
    y <- calcNormFactors(y)
    design <- model.matrix(~0+group, data = y$samples)
    colnames(design) <- levels(y$samples$group)
    y <- estimateDisp(y, design = design, robust = TRUE)
    fit <- glmQLFit(y, design = design)
    
    n_levels <- length(levels(y$samples$group))
    contrast_base <- rep(-(1.0 / (n_levels - 1)), n_levels)
    for (i in 1:n_levels) {
        contrast <- contrast_base
        contrast[i] <- 1
        level <- levels(y$samples$group)[i]
        qlf <- glmQLFTest(fit, contrast = contrast)
        result <- topTags(qlf, n = Inf, sort.by = "PValue")$table
        write.table(
            result, sprintf("%s/%s_result.tsv", subclass_result_dir, level), 
            col.names = TRUE, row.names = TRUE, sep="\t", quote = FALSE
        )
    }
    write.table(
        rownames(y), sprintf("%s/%s_filtered.tsv", subclass_result_dir, subclass), 
        col.names = FALSE, row.names = FALSE, sep="\t", quote = FALSE
    )
}

[1] "Astro"


Removing 111990 rows with all zero counts



[1] "Chandelier"


Removing 21824 rows with all zero counts



[1] "Endo"


...

Removing 18409 rows with all zero counts



[1] "VIP"


Removing 19034 rows with all zero counts



[1] "VLMC"


Removing 39478 rows with all zero counts

